In [2]:
%load_ext autoreload
%autoreload 2
import sys
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
from os.path import join as oj
import pandas as pd
import pickle as pkl
import sklearn.model_selection
import models
import util
import os
import config
import viz
import scipy.stats
from tqdm import tqdm
import figs
import matplotlib.image as mpimg
import seaborn as sns
import face_recognition
import dlib
from PIL import Image
from skimage import io
import scipy.misc
import matplotlib.pyplot as plt

# crop and detect faces in real ims

In [18]:
def detect_faces(image):

    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()

    # Run detector and get bounding boxes of the faces on image.
    detected_faces = face_detector(image, 1)
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]

    return face_frames

# Load image
def crop_and_resize_ims(dir_base='../data/pilot-our-images/',
                        dir_out_base='../data_processed/pilot-our-images-large',
                        MULT = 1.6):
    for folder in os.listdir(dir_base):
        dir_in = oj(dir_base, folder)
        dir_out = oj(dir_out_base, folder)
        os.makedirs(dir_out, exist_ok=True)

        im_num = 0
        for fname in sorted(os.listdir(dir_in)):
            if '.jpg' in fname:
                img_path = oj(dir_in, fname)
                image = io.imread(img_path)

                # Detect faces
                detected_faces = detect_faces(image)
                if len(detected_faces) > 0:

                    # Crop faces and plot
                    (left, upper, right, lower) = detected_faces[0]
                    
                    mid = (left + right) // 2
                    diff = (mid - left) * MULT
                    left = mid - diff
                    right = mid + diff
                    
                    mid = (lower + upper) // 2
                    diff = (mid - lower) * MULT
                    upper = mid + diff
                    lower = mid - diff                   

                    im_face = Image.fromarray(image).crop((left, upper, right, lower))
                    dim_min = min(im_face.width, im_face.height)
                    im_face = im_face.crop((0, 0, dim_min, dim_min)) # im_face[:dim_min, :dim_min]
                    im_face = im_face.resize((1024, 1024))
                    
                    '''
                    util.imshow(np.asarray(im_face))
                    plt.show()
                    '''

                    
                    # save
                    out_fname = oj(dir_out, f'{folder}_{im_num:03d}.jpg')
                    im_face.save(out_fname)
                    im_num += 1

crop_and_resize_ims()        

# lower res ims

In [7]:
sys.path.append('stylegan2encoder')

In [ ]:
N_IMS = 100
out_dir = ''
for fname in sorted(os.listdir(config.DIR_CELEBA_IMS))[N_IMS]:
    

In [9]:
import projector
import pretrained_networks

In [ ]:
network_pkl='gdrive:networks/stylegan2-ffhq-config-f.pkl'
vgg16_pkl='https://drive.google.com/uc?id=1N2-m9qszOeVC9Tq77WxsLnuWwOedQiD2'
num_steps=1000
initial_learning_rate=0.1
initial_noise_factor=0.05
verbose=False
regularize_mean_deviation_weight=0.1
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)

Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...


In [ ]:
print('Loading networks from "%s"...' % network_pkl)

proj = projector.Projector(
    vgg16_pkl             = vgg16_pkl,
    num_steps             = num_steps,
    initial_learning_rate = initial_learning_rate,
    initial_noise_factor  = initial_noise_factor,
    verbose               = verbose,
    regularize_mean_deviation_weight = regularize_mean_deviation_weight
)
proj.set_network(Gs)

In [ ]:

# parser = argparse.ArgumentParser(description='Project real-world images into StyleGAN2 latent space')
# parser.add_argument('src_dir', help='Directory with aligned images for projection')
# parser.add_argument('dst_dir', help='Output directory')
# parser.add_argument('--tmp-dir', default='.stylegan2-tmp', help='Temporary directory for tfrecords and video frames')
# parser.add_argument('--network-pkl', default='gdrive:networks/stylegan2-ffhq-config-f.pkl', help='StyleGAN2 network pickle filename')
# parser.add_argument('--vgg16-pkl', default='https://drive.google.com/uc?id=1N2-m9qszOeVC9Tq77WxsLnuWwOedQiD2', help='VGG16 network pickle filename')
# parser.add_argument('--num-steps', type=int, default=1000, help='Number of optimization steps')
# parser.add_argument('--initial-learning-rate', type=float, default=0.1, help='Initial learning rate')
# parser.add_argument('--initial-noise-factor', type=float, default=0.05, help='Initial noise factor')
# parser.add_argument('--verbose', type=bool, default=False, help='Verbose output')
# parser.add_argument('--video', type=bool, default=False, help='Render video of the optimization process')
# parser.add_argument('--video-mode', type=int, default=1, help='Video mode: 1 for optimization only, 2 for source + optimization')
# parser.add_argument('--video-size', type=int, default=1024, help='Video size (height in px)')
# parser.add_argument('--video-fps', type=int, default=25, help='Video framerate')
# parser.add_argument('--video-codec', default='libx264', help='Video codec')
# parser.add_argument('--video-bitrate', default='5M', help='Video bitrate')



# parser.add_argument('--regularize_mean_deviation_weight', type=float, default=0, help='Penalize different w vectors to be the same')


src_files = sorted([os.path.join(args.src_dir, f) for f in os.listdir(args.src_dir) if f[0] not in '._'])
for src_file in src_files:
    # check if file already exists and skip
    filename = os.path.join(args.dst_dir, os.path.basename(src_file)[:-4] + '.png')
    if not os.path.exists(filename):

        project_image(proj, src_file, args.dst_dir, args.tmp_dir, video=args.video)
        if args.video:
            render_video(
                src_file, args.dst_dir, args.tmp_dir, args.num_steps, args.video_mode,
                args.video_size, args.video_fps, args.video_codec, args.video_bitrate
            )
        shutil.rmtree(args.tmp_dir)
    else:
        print('skipping', filename)